# PART 1 – DATA

a- Create an SQL query that provides one of our main business KPIs - the number of Monthly
Active Users (MAU) per day per offer and per country
    
    A user is defined as Active if he streamed at least one track longer than 30 seconds
    
    MAU are based on a rolling 30 days period

b- Create an SQL query that provides the list of the 10 best streamers of Metronomy in 2016 in
France

c- Create an SQL query that provides the number of days between the inscription and the 100th
stream of all the Brazilian users registered in January 2018

# Import Libraries

In [4]:
import sqlite3
import pandas as pd

# Connection to the database

In [5]:
con = sqlite3.connect('music.db')

# SELECT on TABLES

In [7]:
df = pd.read_sql_query("select * from ALBUMS;", con)

In [8]:
df

,Album_id,Album_title,Creation_date
0,86885,Unknown Pleasures [Collector's Edition],2007-09-10
1,86907,Closer [Collector's Edition],2007-09-10
2,93066,The Best Of,2008-03-20
3,257871,Still [Collector's Edition],2008-03-25
4,296724,Radio Ladio (Remixes),2009-04-06
5,392275,Not Made For Love,2009-09-14
6,423610,Pip Paine (Pay The £5000 You Owe),2009-11-02
7,936365,The English Riviera,2011-04-11
8,1126062,TOTAL,2011-06-03
9,4454911,Closer,2007-09-10
